In [2]:
import pandas as pd

In [3]:
df_price = pd.read_csv('get_around_pricing_project.csv', index_col=0)
df_price.shape

(4843, 14)

In [5]:
df_price.head()

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


Notre target est 'rental_price_per_day' et les autres colonnes sont des variables explicatives

In [6]:
df_price.isna().sum()

model_key                    0
mileage                      0
engine_power                 0
fuel                         0
paint_color                  0
car_type                     0
private_parking_available    0
has_gps                      0
has_air_conditioning         0
automatic_car                0
has_getaround_connect        0
has_speed_regulator          0
winter_tires                 0
rental_price_per_day         0
dtype: int64

In [7]:
df_price['model_key'].value_counts()

model_key
Citroën        969
Renault        916
BMW            827
Peugeot        642
Audi           526
Nissan         275
Mitsubishi     231
Mercedes        97
Volkswagen      65
Toyota          53
SEAT            46
Subaru          44
Opel            33
Ferrari         33
PGO             33
Maserati        18
Suzuki           8
Porsche          6
Ford             5
KIA Motors       3
Alfa Romeo       3
Fiat             2
Lexus            2
Lamborghini      2
Mini             1
Mazda            1
Honda            1
Yamaha           1
Name: count, dtype: int64

In [8]:
df_price['fuel'].value_counts()

fuel
diesel           4641
petrol            191
hybrid_petrol       8
electro             3
Name: count, dtype: int64

In [9]:
df_price['paint_color'].value_counts()

paint_color
black     1633
grey      1175
blue       710
white      538
brown      341
silver     329
red         52
beige       41
green       18
orange       6
Name: count, dtype: int64

In [10]:
df_price['car_type'].value_counts()

car_type
estate         1606
sedan          1168
suv            1058
hatchback       699
subcompact      117
coupe           104
convertible      47
van              44
Name: count, dtype: int64

In [6]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import 

In [12]:
df_price.columns

Index(['model_key', 'mileage', 'engine_power', 'fuel', 'paint_color',
       'car_type', 'private_parking_available', 'has_gps',
       'has_air_conditioning', 'automatic_car', 'has_getaround_connect',
       'has_speed_regulator', 'winter_tires', 'rental_price_per_day'],
      dtype='object')

In [13]:
ohe_columns = ['model_key', 'fuel', 'paint_color', 'car_type']
scaled_columns = ['mileage', 'engine_power']

X = df_price.drop('rental_price_per_day', axis=1)
y = df_price['rental_price_per_day']

In [8]:
ohe_pipe = Pipeline(
    steps = [
        #('imputer', KNNImputer()),
        ('ohe', OneHotEncoder())
    ]
)

scaler_pipe = Pipeline(
    steps= [
       #('imputer', SimpleImputer()),
       ('scaler', StandardScaler()) 
    ]
)

preprocessing = ColumnTransformer(
    transformers = [
        ('ohe', ohe_pipe, ohe_columns),
        ('standard', scaler_pipe, scaled_columns)
    ],
    remainder = 'passthrough'
)

ml_pipe = Pipeline(
    steps = [
        ('preprocessing', preprocessing),
        ('model', LinearRegression())
    ]
)

In [15]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Define the pipelines for preprocessing
ohe_pipe = Pipeline(
    steps=[
        ('ohe', OneHotEncoder(handle_unknown='ignore'))
    ]
)

scaler_pipe = Pipeline(
    steps=[
        ('scaler', StandardScaler())
    ]
)

# Combine the preprocessing steps
preprocessing = ColumnTransformer(
    transformers=[
        ('ohe', ohe_pipe, ohe_columns),
        ('standard', scaler_pipe, scaled_columns)
    ],
    remainder='passthrough'
)

# Define the main pipeline with XGBoost Regressor
ml_pipe = Pipeline(
    steps=[
        ('preprocessing', preprocessing),
        ('model', XGBRegressor(objective='reg:squarederror'))
    ]
)

param_grid = {
    'model__n_estimators': [50, 100, 200],
    'model__learning_rate': [0.01, 0.1],
    'model__max_depth': [3, 5, 7],
    'model__max_leaves': [0, 31, 50],
    'model__subsample': [0.8, 1.0],
    'model__num_parallel_tree': [1, 5, 10]
}

# Setup GridSearchCV
grid_search = GridSearchCV(ml_pipe, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe',
                                                                         Pipeline(steps=[('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['model_key',
                                                                          'fuel',
                                                                          'paint_color',
                                                                          'car_type']),
                                                                        ('standard',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['mileage',
                                                                          'engine_power'])])),
                                       ('model',
                                        XGBRegressor(bas...
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=None, ...))]),
             n_jobs=-1,
             param_grid={'model__learning_rate': [0.01, 0.1],
                         'model__max_depth': [3, 5, 7],
                         'model__max_leaves': [0, 31, 50],
                         'model__n_estimators': [50, 100, 200],
                         'model__num_parallel_tree': [1, 5, 10],
                         'model__subsample': [0.8, 1.0]},
             scoring='neg_mean_squared_error', verbose=1)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
grid_search.fit(X_train,y_train)

ValueError: could not convert string to float: 'Renault'

In [35]:
x = {
'model_key': ['Renault'],
'mileage': [109839],
'engine_power': [135],
'fuel': ['diesel'],
'paint_color': ['black'],
'car_type': ['sedan'],
'private_parking_available': [True],
'has_gps': [True],
'has_air_conditioning': [False],
'automatic_car': [False],
'has_getaround_connect': [True],
'has_speed_regulator': [False],
'winter_tires': [True],
}

pd.DataFrame(x)

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires
0,Renault,109839,135,diesel,black,sedan,True,True,False,False,True,False,True


In [ ]:
model.predict(x)